# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [3]:
using Pkg
Pkg.add("SatelliteDynamics")  # 최초 1회만 설치
using SatelliteDynamics

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed URIParser ───────── v0.4.1
   Installed Suppressor ──────── v0.2.8
   Installed BinDeps ─────────── v1.0.2
   Installed SOFA ────────────── v1.0.2
   Installed SatelliteDynamics ─ v0.5.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [0e7c1a32] + SatelliteDynamics v0.5.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [9e28174c] + BinDeps v1.0.2
  [ad3d3fd0] + SOFA v1.0.2
  [0e7c1a32] + SatelliteDynamics v0.5.0
  [fd094767] + Suppressor v0.2.8
  [30578b45] + URIParser v0.4.1
    Building SOFA → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/2d2697ca362f3fb16f3a129881f9442e9bdc1ecf/build.log`
Precompiling packages...
   3365.8 ms  ✓ Suppressor
   4770.7 ms  ✓ URIParser
   1186.4 ms  ✓ BinDeps
   1368.6 ms  ✓ SOFA
   5373.4 ms  ✓ SatelliteDynamics
  5 dependencies successfully precompiled in 21 seconds. 459 already precompiled.


In [10]:
using SatelliteDynamics

# 위성의 ECI 위치/속도 벡터
r_eci = [6879.788e3, 0.000001e3, 0.000006e3]
v_eci = [-0.04346e3, 5.477441e3, 5.289504e3]
delv_eci = [0.3, 0, 0]
state_eci = vcat(r_eci, v_eci)

# ECI → LVLH(RTN) 변환 회전행렬 계산
R_eci2lvlh = rECItoRTN(state_eci)

# 변환하고자 하는 벡터 (예: 위성의 ECI 위치)
delv_lvlh = R_eci2lvlh * delv_eci

println("LVLH 좌표: ", delv_lvlh)
print(R_eci2lvlh)


LVLH 좌표: [0.3, -2.1311515412506564e-10, -1.579138820701483e-10]
[1.0 1.453533161196246e-10 8.721198967177477e-10; -7.103838470835522e-10 0.719339753636532 0.6946584188204542; -5.263796069004944e-10 -0.6946584188204542 0.719339753636532]

In [ ]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />